In [1]:
import pandas as pd
import xgboost as xgb
import gc

In [2]:
from utils import *

In [16]:
mname = 'xgb'

In [3]:
df_train, df_val, df_sub = load()

df_train.shape 161570557,13
df_val.shape 23333333,13
df_sub.shape 18790469,13


In [4]:
data_val = xgb.DMatrix(df_val[xnames], df_val[yname])
data = xgb.DMatrix(df_train[xnames], df_train[yname])

In [5]:
del df_train, df_val
gc.collect()

72

In [6]:
params = {'eta': 0.6,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 31,  
          'max_depth': 5, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'nthread': 16,
          'objective': 'binary:logistic', 
          'scale_pos_weight':99,
          'eval_metric': 'auc', 
          'random_state': 2333}

In [7]:
r = xgb.train(params,
                  data,
                  300,
                  [(data, 'train'), (data_val, 'valid')],
                  maximize=True,
                  early_stopping_rounds = 25)

[0]	train-auc:0.89734	valid-auc:0.903754
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 25 rounds.
[1]	train-auc:0.932559	valid-auc:0.934295
[2]	train-auc:0.938943	valid-auc:0.939565


In [15]:
from sklearn.externals import joblib
joblib.dump(r, '../model/%s.pkl' % mname) 

['../model/xgb.pkl']

In [8]:
y_val_pred = r.predict(data_val, ntree_limit=r.best_ntree_limit)

In [17]:
import pickle
print('writing val ...')
output_val = pd.DataFrame({'y_pred': y_val_pred, 'y': data_val.get_label()})
with open('../data/%s_val.pickle' % mname, 'wb') as handle:
    pickle.dump(output_val, handle, protocol=pickle.HIGHEST_PROTOCOL)

writing val ...


In [18]:
ids = df_sub['click_id'].astype(int).values
X_sub = xgb.DMatrix(df_sub[xnames])
print('predicting...')
y_sub = r.predict(X_sub, ntree_limit=r.best_ntree_limit)
print('writing...')
output = pd.DataFrame({'click_id': ids, 'is_attributed': y_sub})
output.to_csv("../data/%s.csv.gz" % mname, compression='gzip', index=False)

predicting...
writing...
